In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import time

### Dataset:

In [2]:
try:
    from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator
except:
    !git clone https://github.com/airi-industrial-ai/fddbenchmark
    from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator

In [3]:
tep_dataset = FDDDataset(name='reinartz_tep', splitting_type='supervised')
tep_dataset.df = (tep_dataset.df - tep_dataset.df.mean())/tep_dataset.df.std()
tep_dataset.df.head()

Reading data/reinartz_tep/test_mask.csv: 100%|██████████| 5600000/5600000 [00:01<00:00, 3153162.00it/s]


xmeas_1   xmeas_2   xmeas_3   xmeas_4   xmeas_5   xmeas_6  \
run_id     sample                                                               
1334009671 1      -0.175458  0.097509 -0.092930  0.022772  0.079000  0.056226   
           2      -0.158513 -0.887449 -0.243139  0.211698 -0.352012 -0.553442   
           3      -0.166950  1.864230 -0.384321 -0.192391 -0.540460 -0.496256   
           4      -0.165755 -0.425304 -0.289450  0.085419  1.186139  0.114638   
           5      -0.179773  1.085111 -0.254781  0.393148 -0.116478 -0.816772   

                    xmeas_7   xmeas_8   xmeas_9  xmeas_10  ...     xmv_2  \
run_id     sample                                          ...             
1334009671 1       0.057243  0.078469 -0.042816 -0.081580  ... -0.101671   
           2      -0.006126 -0.248531 -0.358203 -0.098747  ... -0.037165   
           3       0.047474  0.041677  0.116272 -0.115753  ... -0.131093   
           4      -0.020781  0.570426  0.066785 -0.146494  ... -0.067351   
           5       0.051283 -0.513587 -0.061174 -0.184990  ... -0.059039   

                      xmv_3     xmv_4  xmv_5     xmv_6     xmv_7     xmv_8  \
run_id     sample                                                            
1334009671 1      -0.237457 -0.139339    NaN -0.082903  0.236431  0.025629   
           2      -0.234886 -0.141768    NaN -0.105557  0.236835  0.066062   
           3      -0.232738 -0.140334    NaN -0.120353  0.108759 -0.071836   
           4      -0.231545 -0.160266    NaN -0.144987  0.036609 -0.135977   
           5      -0.230173 -0.122039    NaN -0.166149  0.122855  0.056399   

                   xmv_9    xmv_10    xmv_11  
run_id     sample                             
1334009671 1         NaN -0.149639 -1.448923  
           2         NaN -0.203725 -0.178945  
           3         NaN  0.048110  0.073242  
           4         NaN -0.178125  0.081174  
           5         NaN -0.243887  1.057207  

[5 rows x 52 columns]

In [4]:
tep_dataset.df['xmv_5'] = 1.0
tep_dataset.df['xmv_9'] = 1.0

### Dataloader:

In [5]:
train_dl = FDDDataloader(
    dataframe=tep_dataset.df,
    mask=tep_dataset.train_mask,
    labels=tep_dataset.labels,
    window_size=10,
    step_size=1,
    minibatch_training=True,
    batch_size=512,
    shuffle=True
)

test_dl = FDDDataloader(
    dataframe=tep_dataset.df,
    mask=tep_dataset.test_mask,
    labels=tep_dataset.labels,
    window_size=10, 
    step_size=1, 
    minibatch_training=True,
    batch_size=512
)

Creating sequence of samples: 100%|██████████| 560/560 [00:24<00:00, 23.31it/s]


### Model:

In [6]:
num_nodes = 52 # number of nodes
window_size = 10 # window size
nhidden = 32 # feature size in GCN layers
out_channels = 29 # number of process states

In [7]:
from models import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

adj = torch.load('direct_A.pt')
model = SimpleGNNTEP(num_nodes, window_size, nhidden, out_channels, device, adj)

### Training:

In [8]:
model.to(device)
optimizer = Adam(model.parameters(), lr=0.001)
n_epochs = 50

In [ ]:
%%time
for e in range(n_epochs):
    model.train()
    av_loss = []
    for train_ts, train_index, train_label in train_dl:
        m = torch.FloatTensor(train_ts)
        v_train_ts = torch.transpose(m, 1, 2)
        train_label = torch.LongTensor(train_label.values).to(device)
        weight = torch.ones(29) * 0.5
        weight[1:] /= 20
        logits = model(v_train_ts)
        loss = F.cross_entropy(logits, train_label, weight=weight.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        av_loss.append(loss.item())
    print(f'Epoch: {e+1:2d}/{n_epochs}, average CE loss: {sum(av_loss)/len(av_loss):.4f}')
    
    preds = []
    test_labels = []
    model.eval()
    for test_ts, test_index, test_label in test_dl:
        m = torch.FloatTensor(test_ts)
        v_test_ts = torch.transpose(m, 1, 2)
        with torch.no_grad():
            logits = model(v_test_ts)
        pred = logits.argmax(axis=1).cpu().numpy()
        preds.append(pd.Series(pred, index=test_index))
        test_labels.append(test_label)
    pred = pd.concat(preds)
    test_label = pd.concat(test_labels)
    
    evaluator = FDDEvaluator(
    splitting_type=tep_dataset.splitting_type, 
    step_size=test_dl.step_size
    )
    evaluator.print_metrics(test_label, pred)
    
    torch.save(model, 'directed_epoch_'+str(e+1)+'.pth')

Epoch:  1/50, average CE loss: 0.2429
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9889/0.0000
    Fault 02: 0.9840/0.0000
    Fault 03: 0.6579/0.0000
    Fault 04: 0.9920/0.0000
    Fault 05: 0.2832/0.0010
    Fault 06: 0.9927/0.0000
    Fault 07: 0.9979/0.0000
    Fault 08: 0.8976/0.0000
    Fault 09: 0.0930/0.0001
    Fault 10: 0.9501/0.0000
    Fault 11: 0.9040/0.0000
    Fault 12: 0.8450/0.0001
    Fault 13: 0.8205/0.0000
    Fault 14: 0.9912/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0000/0.0000
    Fault 17: 0.9730/0.0000
    Fault 18: 0.9077/0.0000
    Fault 19: 0.9693/0.0000
    Fault 20: 0.9660/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.3041/0.0000
    Fault 23: 0.0082/0.0000
    Fault 24: 0.9363/0.0000
    Fault 25: 0.9373/0.0000
    Fault 26: 0.9247/0.0000
    Fault 27: 0.9632/0.0000
    Fault 28: 0.1989/0.0004
Detection TPR: 0.7411
Detection FPR: 0.0017
Average Detection Delay (ADD): 69.01
Total Correct Diagnosis Rate (Total CDR): 0.9391

Clusteri

Epoch:  9/50, average CE loss: 0.1617
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9938/0.0000
    Fault 02: 0.9886/0.0000
    Fault 03: 0.8477/0.0001
    Fault 04: 0.9922/0.0000
    Fault 05: 0.3020/0.0009
    Fault 06: 0.9987/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9405/0.0000
    Fault 09: 0.2165/0.0002
    Fault 10: 0.9633/0.0000
    Fault 11: 0.9346/0.0000
    Fault 12: 0.8798/0.0002
    Fault 13: 0.9309/0.0000
    Fault 14: 0.9962/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0063/0.0000
    Fault 17: 0.9798/0.0000
    Fault 18: 0.9456/0.0000
    Fault 19: 0.9840/0.0000
    Fault 20: 0.9676/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.4038/0.0000
    Fault 23: 0.1397/0.0001
    Fault 24: 0.9684/0.0000
    Fault 25: 0.9524/0.0000
    Fault 26: 0.9482/0.0000
    Fault 27: 0.9667/0.0000
    Fault 28: 0.2386/0.0004
Detection TPR: 0.7716
Detection FPR: 0.0020
Average Detection Delay (ADD): 58.64
Total Correct Diagnosis Rate (Total CDR): 0.9481

Clusteri

Epoch: 17/50, average CE loss: 0.1466
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9969/0.0000
    Fault 02: 0.9875/0.0000
    Fault 03: 0.8401/0.0001
    Fault 04: 0.9938/0.0000
    Fault 05: 0.2904/0.0008
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9264/0.0000
    Fault 09: 0.2111/0.0004
    Fault 10: 0.9639/0.0000
    Fault 11: 0.9400/0.0000
    Fault 12: 0.8858/0.0001
    Fault 13: 0.9489/0.0000
    Fault 14: 0.9964/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0074/0.0000
    Fault 17: 0.9796/0.0000
    Fault 18: 0.9476/0.0000
    Fault 19: 0.9820/0.0000
    Fault 20: 0.9674/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.4436/0.0000
    Fault 23: 0.7140/0.0000
    Fault 24: 0.9714/0.0000
    Fault 25: 0.9530/0.0000
    Fault 26: 0.9497/0.0000
    Fault 27: 0.9524/0.0000
    Fault 28: 0.2234/0.0003
Detection TPR: 0.7939
Detection FPR: 0.0019
Average Detection Delay (ADD): 47.70
Total Correct Diagnosis Rate (Total CDR): 0.9479

Clusteri

Epoch: 25/50, average CE loss: 0.1440
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9969/0.0000
    Fault 02: 0.9874/0.0000
    Fault 03: 0.8285/0.0001
    Fault 04: 0.9943/0.0000
    Fault 05: 0.2914/0.0008
    Fault 06: 0.9992/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9237/0.0000
    Fault 09: 0.2128/0.0004
    Fault 10: 0.9652/0.0000
    Fault 11: 0.9430/0.0000
    Fault 12: 0.8901/0.0001
    Fault 13: 0.9474/0.0000
    Fault 14: 0.9964/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0110/0.0000
    Fault 17: 0.9798/0.0000
    Fault 18: 0.9446/0.0000
    Fault 19: 0.9817/0.0000
    Fault 20: 0.9685/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.4486/0.0000
    Fault 23: 0.8095/0.0000
    Fault 24: 0.9750/0.0000
    Fault 25: 0.9575/0.0000
    Fault 26: 0.9544/0.0000
    Fault 27: 0.9544/0.0000
    Fault 28: 0.2202/0.0003
Detection TPR: 0.7981
Detection FPR: 0.0018
Average Detection Delay (ADD): 49.88
Total Correct Diagnosis Rate (Total CDR): 0.9478

Clusteri

Epoch: 33/50, average CE loss: 0.1427
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9971/0.0000
    Fault 02: 0.9892/0.0000
    Fault 03: 0.8448/0.0001
    Fault 04: 0.9945/0.0000
    Fault 05: 0.2880/0.0007
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9985/0.0000
    Fault 08: 0.9188/0.0000
    Fault 09: 0.2157/0.0003
    Fault 10: 0.9655/0.0000
    Fault 11: 0.9410/0.0000
    Fault 12: 0.8868/0.0001
    Fault 13: 0.9497/0.0000
    Fault 14: 0.9965/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0140/0.0000
    Fault 17: 0.9797/0.0000
    Fault 18: 0.9467/0.0000
    Fault 19: 0.9821/0.0000
    Fault 20: 0.9685/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.4341/0.0000
    Fault 23: 0.8421/0.0000
    Fault 24: 0.9752/0.0000
    Fault 25: 0.9598/0.0000
    Fault 26: 0.9542/0.0000
    Fault 27: 0.9593/0.0000
    Fault 28: 0.2372/0.0003
Detection TPR: 0.7990
Detection FPR: 0.0016
Average Detection Delay (ADD): 48.47
Total Correct Diagnosis Rate (Total CDR): 0.9494

Clusteri

Epoch: 41/50, average CE loss: 0.1416
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9972/0.0000
    Fault 02: 0.9894/0.0000
    Fault 03: 0.8747/0.0001
    Fault 04: 0.9958/0.0000
    Fault 05: 0.2707/0.0006
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9985/0.0000
    Fault 08: 0.9294/0.0000
    Fault 09: 0.2343/0.0003
    Fault 10: 0.9641/0.0000
    Fault 11: 0.9398/0.0000
    Fault 12: 0.8836/0.0002
    Fault 13: 0.9463/0.0000
    Fault 14: 0.9965/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0190/0.0000
    Fault 17: 0.9796/0.0000
    Fault 18: 0.9483/0.0000
    Fault 19: 0.9804/0.0000
    Fault 20: 0.9686/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.4056/0.0000
    Fault 23: 0.8215/0.0000
    Fault 24: 0.9762/0.0000
    Fault 25: 0.9613/0.0000
    Fault 26: 0.9549/0.0000
    Fault 27: 0.9599/0.0000
    Fault 28: 0.2602/0.0003
Detection TPR: 0.7987
Detection FPR: 0.0016
Average Detection Delay (ADD): 45.92
Total Correct Diagnosis Rate (Total CDR): 0.9504

Clusteri

Epoch: 49/50, average CE loss: 0.1408
FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9974/0.0000
    Fault 02: 0.9889/0.0000
    Fault 03: 0.8858/0.0001
    Fault 04: 0.9956/0.0000
    Fault 05: 0.2732/0.0006
    Fault 06: 0.9992/0.0000
    Fault 07: 0.9986/0.0000
    Fault 08: 0.9329/0.0000
    Fault 09: 0.2071/0.0002
    Fault 10: 0.9645/0.0000
    Fault 11: 0.9380/0.0000
    Fault 12: 0.8810/0.0001
    Fault 13: 0.9520/0.0000
    Fault 14: 0.9964/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0196/0.0000
    Fault 17: 0.9796/0.0000
    Fault 18: 0.9496/0.0000
    Fault 19: 0.9790/0.0000
    Fault 20: 0.9688/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.4222/0.0000
    Fault 23: 0.8470/0.0000
    Fault 24: 0.9761/0.0000
    Fault 25: 0.9617/0.0000
    Fault 26: 0.9566/0.0000
    Fault 27: 0.9665/0.0000
    Fault 28: 0.2688/0.0004
Detection TPR: 0.7995
Detection FPR: 0.0015
Average Detection Delay (ADD): 49.33
Total Correct Diagnosis Rate (Total CDR): 0.9518

Clusteri